In [1]:

# 导入相关的库
import numpy as np
import pandas as pd
import statsmodels.api as sm
import alphalens as al
import warnings 
import pandas as pd

# 从CSV文件读取量价数据
file_path = 'e:temp/price.csv'   # 文件的路径和文件名，注意：此处要换成你的文件地址
price_df = pd.read_csv(file_path, encoding='gbk', index_col=0) 
price_df['trade_date'] = pd.to_datetime(price_df['trade_date'])  # 将日期统一为datetime格式

# 计算动量因子
price_df['MOM_10'] = price_df.groupby('stock_code')['close'].pct_change(10)  # 10天的mom因子
price_df['MOM_100'] = price_df.groupby('stock_code')['close'].pct_change(100)  # 100天的mom因子
price_df['MOM_200'] = price_df.groupby('stock_code')['close'].pct_change(200)  # 200天的mom因子

# 选取2010年之后的数据
df = price_df[price_df['trade_date']>='2010-01-01']  
df = df[['trade_date', 'stock_code', 'open', 'MOM_10', 'MOM_100', 'MOM_200']].reset_index(drop=True)

# 关闭警告信息
warnings.filterwarnings('ignore')

# 从csv文件读取数据  
file_path = 'F:/quatntData/growth/growth_factor.csv'
data_df = pd.read_csv(file_path, encoding='gbk', index_col=0) 
data_df['日期'] = pd.to_datetime(data_df['日期'])  # 将日期统一为datetime格式

# 设置将要评价的因子
factor_name = '营业收入同比增长率'

# 生成符合Alphalens要求格式的因子值数据

factor = data_df.set_index(['日期','股票代码'])[factor_name]
# 生成符合Alphalens要求格式的交易价格数据
prices = data_df.pivot(index='日期', columns='股票代码', values='开盘价')
prices = prices.shift(-1)  # 将第二天的开盘价作为交易价格，避免用到“未来数据”

# 预处理因子数据，得到符合Alphalens需要的数据格式。
factor_data = al.utils.get_clean_factor_and_forward_returns(
    factor=factor,
    prices=prices,
    quantiles=10,
    periods=(1, 10))
    
# 生成因子性能报告
al.tears.create_full_tear_sheet(factor_data)

KeyboardInterrupt: 